In [1]:
#Library install

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
from sklearn.utils import resample
np.random.seed(42)
from sklearn.metrics import confusion_matrix

import keras
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.preprocessing import LabelEncoder

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [5]:
# level encoding for convert the string values to numeric

skin_data = pd.read_csv('../input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')
SIZE=32

In [6]:
# label encoding to numeric values from text
l = LabelEncoder()
l.fit(skin_data['dx'])
LabelEncoder()
print(list(l.classes_))
 
skin_data['label'] = l.transform(skin_data["dx"]) 

['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']


In [7]:
df_0 = skin_data[skin_data['label'] == 0]
df_1 = skin_data[skin_data['label'] == 1]
df_2 = skin_data[skin_data['label'] == 2]
df_3 = skin_data[skin_data['label'] == 3]
df_4 = skin_data[skin_data['label'] == 4]
df_5 = skin_data[skin_data['label'] == 5]
df_6 = skin_data[skin_data['label'] == 6]


In [8]:
# Sampling the datas such that for each output class the training set contains 
# 400 images
n_samples=400 
df_0_balanced = resample(df_0, replace=True, n_samples=n_samples, random_state=42) 
df_1_balanced = resample(df_1, replace=True, n_samples=n_samples, random_state=42) 
df_2_balanced = resample(df_2, replace=True, n_samples=n_samples, random_state=42)
df_3_balanced = resample(df_3, replace=True, n_samples=n_samples, random_state=42)
df_4_balanced = resample(df_4, replace=True, n_samples=n_samples, random_state=42)
df_5_balanced = resample(df_5, replace=True, n_samples=n_samples, random_state=42)
df_6_balanced = resample(df_6, replace=True, n_samples=n_samples, random_state=42)


In [9]:
# Construct a balanced dataset
skin_data_balanced = pd.concat([df_0_balanced, df_1_balanced, 
                              df_2_balanced, df_3_balanced, 
                              df_4_balanced, df_5_balanced, df_6_balanced])

In [10]:
base_skin_dir = '../input/skin-cancer-mnist-ham10000'
image_path = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_skin_dir, '*', '*.jpg'))}
skin_data_balanced['path'] = skin_data['image_id'].map(image_path.get)

In [11]:
base_skin_dir = '../input/skin-cancer-mnist-ham10000'
image_path = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_skin_dir, '*', '*.jpg'))}
skin_data_balanced['path'] = skin_data['image_id'].map(image_path.get)
#Use the path to read images.
skin_data_balanced['image'] = skin_data_balanced['path'].map(lambda x:
np.asarray(Image.open(x).resize((SIZE,SIZE))))

In [12]:
# Scaling and the train test split
X = np.asarray(skin_data_balanced['image'].tolist())
X = X/255.
Y=skin_data_balanced['label']  
Y_cat = to_categorical(Y, num_classes=7)
x_train, x_test, y_train, y_test = train_test_split(X, Y_cat, test_size=0.25, random_state=42)

In [13]:
# Adding layer to CNN model.
num_classes = 7

model = Sequential()
model.add(Conv2D(256, (3, 3), activation="relu", input_shape=(SIZE, SIZE, 3)))
#model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))  
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3),activation='relu'))
#model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))  
model.add(Dropout(0.3))

model.add(Conv2D(64, (3, 3),activation='relu'))
#model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))  
model.add(Dropout(0.3))
model.add(Flatten())

model.add(Dense(32))
model.add(Dense(7, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['acc'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 256)       7168      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 256)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 15, 15, 256)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 128)       295040    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 128)        0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 6, 6, 128)         0

In [14]:
# Result

batch_size = 16
epochs = 60
history = model.fit(
    x_train, y_train,
    epochs=epochs,
    batch_size = batch_size,
    validation_data=(x_test, y_test),
    verbose=2)

score = model.evaluate(x_test, y_test)
print('The test accuracy is:', score[1])

Epoch 1/60
132/132 - 15s - loss: 1.9150 - acc: 0.1938 - val_loss: 1.7842 - val_acc: 0.2829 - 15s/epoch - 110ms/step
Epoch 2/60
132/132 - 12s - loss: 1.7866 - acc: 0.2490 - val_loss: 1.6599 - val_acc: 0.3257 - 12s/epoch - 94ms/step
Epoch 3/60
132/132 - 13s - loss: 1.6046 - acc: 0.3324 - val_loss: 1.5117 - val_acc: 0.3671 - 13s/epoch - 96ms/step
Epoch 4/60
132/132 - 12s - loss: 1.4954 - acc: 0.3914 - val_loss: 1.4327 - val_acc: 0.4271 - 12s/epoch - 92ms/step
Epoch 5/60
132/132 - 12s - loss: 1.4467 - acc: 0.4238 - val_loss: 1.3698 - val_acc: 0.4714 - 12s/epoch - 93ms/step
Epoch 6/60
132/132 - 13s - loss: 1.4093 - acc: 0.4424 - val_loss: 1.3928 - val_acc: 0.4729 - 13s/epoch - 96ms/step
Epoch 7/60
132/132 - 12s - loss: 1.3333 - acc: 0.4686 - val_loss: 1.3657 - val_acc: 0.4457 - 12s/epoch - 92ms/step
Epoch 8/60
132/132 - 13s - loss: 1.3197 - acc: 0.4738 - val_loss: 1.2624 - val_acc: 0.5043 - 13s/epoch - 95ms/step
Epoch 9/60
132/132 - 12s - loss: 1.2696 - acc: 0.5024 - val_loss: 1.2840 - val_